In [1]:
import pandas as pd
from pandas import Series, DataFrame
import numpy as np
from scipy.stats import mode
import csv
import matplotlib.dates
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import seaborn as sns
from datetime import *
import matplotlib.pylab as pylab
from pylab import *  
mpl.rcParams['font.sans-serif'] = ['SimHei']


In [2]:
#导入临时处理的训练数据集
train_df_all = pd.read_csv('/home/lab-zhao.yinhu/kengkeng/alimama/data/fusai_a_train_df_all.csv')
print(train_df_all.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10432036 entries, 0 to 10432035
Data columns (total 74 columns):
instance_id                              int64
item_id                                  int64
item_brand_id                            int64
item_city_id                             int64
item_price_level                         int64
item_sales_level                         int64
item_collected_level                     int64
item_pv_level                            int64
user_id                                  int64
user_gender_id                           int64
user_age_level                           int64
user_occupation_id                       int64
user_star_level                          int64
context_id                               int64
context_timestamp                        int64
context_page_id                          int64
shop_id                                  int64
shop_review_num_level                    int64
shop_review_positive_rate              

In [3]:
train_df_1 = train_df_all[(train_df_all.day == 31) | (train_df_all.day <= 4)]
train_df_2 = train_df_all[(train_df_all.day == 5) | (train_df_all.day == 6)]
train_df_3 = train_df_all[train_df_all.day == 7]
print(len(train_df_1))
print(len(train_df_2))
print(len(train_df_3))


6220199
3134662
1077175


In [4]:
test_df = pd.read_csv('/home/lab-zhao.yinhu/kengkeng/alimama/data/fusai_a_test_df_all.csv')
print(test_df.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 519888 entries, 0 to 519887
Data columns (total 73 columns):
instance_id                              519888 non-null int64
item_id                                  519888 non-null int64
item_brand_id                            519888 non-null int64
item_city_id                             519888 non-null int64
item_price_level                         519888 non-null int64
item_sales_level                         519888 non-null int64
item_collected_level                     519888 non-null int64
item_pv_level                            519888 non-null int64
user_id                                  519888 non-null int64
user_gender_id                           519888 non-null int64
user_age_level                           519888 non-null int64
user_occupation_id                       519888 non-null int64
user_star_level                          519888 non-null int64
context_id                               519888 non-null int64
context

In [5]:
# 定义获取某种店铺，品牌，城市对应商品种类，用户数的函数
def getCorrespondNumber(train_df_1, train_df_2, train_df_3, test_df, colName1, colName2, newColName):
    
    df = pd.concat([train_df_1[[colName1, colName2, 'instance_id']], train_df_2[[colName1, colName2, 'instance_id']], train_df_3[[colName1, colName2, 'instance_id']], test_df[[colName1, colName2, 'instance_id']]])
    temp_df = df[[colName1, colName2, 'instance_id']]
    tempDf = temp_df.sort_values(by=colName1, ascending=False)
    tempDf['last_' + colName1] = tempDf[colName1].shift(1)
    tempDf['same'] = tempDf['last_' + colName1]==tempDf[colName1]
#     print(tempDf.head(10))
    colName1List = []
    countList = []
    colName2Set = set()
    for same, col2, last_col1 in tempDf[['same', colName2, 'last_' + colName1]].values:
        if same:
            colName2Set.add(col2)
        else:
            colName1List.append(last_col1)
            countList.append(len(colName2Set))
            colName2Set = {col2}
    #处理最后一行数据
    last_col1 = tempDf.iloc[-1][colName1]
    last_count = len(colName2Set)
    colName1List.append(last_col1)
    countList.append(last_count)
    
    #将结果组合到tempDf中
    result_df = {colName1: colName1List, newColName: countList}
    result_df = DataFrame(result_df)
    result_df = result_df[1:]
    
    tempDf = tempDf.drop_duplicates([colName1])
    tempDf[newColName] = result_df[newColName].values
    
    print(len(train_df_1))
    train_df_1 = pd.merge(train_df_1, tempDf[[colName1, newColName]], on=[colName1], how='left')
    print(len(train_df_1))
    train_df_2 = pd.merge(train_df_2, tempDf[[colName1, newColName]], on=[colName1], how='left')
    train_df_3 = pd.merge(train_df_3, tempDf[[colName1, newColName]], on=[colName1], how='left')
    test_df = pd.merge(test_df, tempDf[[colName1, newColName]], on=[colName1], how='left')
    
    return train_df_1, train_df_2, train_df_3, test_df

train_df_1, train_df_2, train_df_3, test_df = getCorrespondNumber(train_df_1, train_df_2, train_df_3, test_df, 'shop_id', 'item_id', 'shop_item_classNumber')



                     shop_id              item_id          instance_id  \
2485970  9222198553207956009  4104109670731530766  8961704812512965025   
988299   9222198553207956009  4104109670731530766  1302287066853296967   
3884278  9222198553207956009  4104109670731530766  1159122887523006666   
9160562  9222198553207956009   497358163568493573  5670635702949230872   
9399412  9222198553207956009  4104109670731530766  6969340851962766610   
3666984  9222198553207956009   497358163568493573  4727296794933997259   
9710946  9222198553207956009   497358163568493573   494404154474765098   
6318609  9222198553207956009  4104109670731530766  5960253679865760142   
1738145  9222198553207956009  4104109670731530766  6208622505374367007   
860314   9222198553207956009   497358163568493573  1849821548800846176   

         last_shop_id   same  
2485970           NaN  False  
988299   9.222199e+18   True  
3884278  9.222199e+18   True  
9160562  9.222199e+18   True  
9399412  9.222199e+18   True  

In [6]:
# 定义获取用户浏览过商品的平均值，众数，中位数，最大值，最小值
def getUserItemStatFuture(train_df_1, train_df_2, train_df_3, test_df, colName):
    df = pd.concat([train_df_1, train_df_2, train_df_3, test_df])
    df_user_item_pivot_table = pd.pivot_table(df, index=['user_id'], values=[colName], aggfunc=[np.mean, np.max, np.min, np.median])
    df_user_item_pivot_table.reset_index(inplace=True)
    df_user_item_pivot_table.columns = ['user_id', colName + '_mean', colName + '_max', colName + '_min', colName + '_median']
    
    df_mode_pivot_table = pd.pivot_table(df[['user_id', colName, 'instance_id']], index=['user_id', colName], values=['instance_id'], aggfunc=len)
    df_mode_pivot_table.reset_index(inplace=True)
    df_mode_pivot_table = df_mode_pivot_table.sort_values(by=['user_id', 'instance_id'], ascending=False)
    df_mode_pivot_table = df_mode_pivot_table.drop_duplicates(['user_id'])
    df_mode_pivot_table.rename(columns={colName:colName + '_mode'}, inplace=True)
    
    train_df_1 = pd.merge(train_df_1, df_user_item_pivot_table, on=['user_id'], how='left')
    train_df_1 = pd.merge(train_df_1, df_mode_pivot_table[['user_id', colName + '_mode']], on=['user_id'], how='left')
    train_df_2 = pd.merge(train_df_2, df_user_item_pivot_table, on=['user_id'], how='left')
    train_df_2 = pd.merge(train_df_2, df_mode_pivot_table[['user_id', colName + '_mode']], on=['user_id'], how='left')
    train_df_3 = pd.merge(train_df_3, df_user_item_pivot_table, on=['user_id'], how='left')
    train_df_3 = pd.merge(train_df_3, df_mode_pivot_table[['user_id', colName + '_mode']], on=['user_id'], how='left')
    test_df = pd.merge(test_df, df_user_item_pivot_table, on=['user_id'], how='left')
    test_df = pd.merge(test_df, df_mode_pivot_table[['user_id', colName + '_mode']], on=['user_id'], how='left')
    return train_df_1, train_df_2, train_df_3, test_df
    
train_df_1, train_df_2, train_df_3, test_df = getUserItemStatFuture(train_df_1, train_df_2, train_df_3, test_df, 'item_price_level')



In [55]:
# 拆分多维度拼接的字段
def splitMultiFea(df):
    tempDf = df.drop_duplicates(subset=['item_id'])[['item_id','item_category_list','item_property_list']]
    tempDf['item_category_list_str'] = tempDf['item_category_list'].values
    tempDf['item_property_list_str'] = tempDf['item_property_list'].values
    tempDf['item_category_list'] = tempDf[tempDf.item_category_list.notnull()]['item_category_list'].map(lambda x: x.split(';'))
    tempDf['item_category0'] = tempDf[tempDf.item_category_list.notnull()]['item_category_list'].map(lambda x: x[0])
    tempDf['item_category1'] = tempDf[tempDf.item_category_list.notnull()]['item_category_list'].map(lambda x: x[1] if len(x)>1 else np.nan)
    tempDf['item_category2'] = tempDf[tempDf.item_category_list.notnull()]['item_category_list'].map(lambda x: x[2] if len(x)>2 else np.nan)
    tempDf['item_property_list'] = tempDf[tempDf.item_property_list.notnull()]['item_property_list'].map(lambda x: x.split(';'))
    df = df.drop(['item_category_list','item_property_list'], axis=1).merge(tempDf, how='left', on='item_id')
    df['item_prop_num'] = df['item_property_list'].dropna().map(lambda x: len(x))
    df['predict_category_property_str'] = df['predict_category_property'].values
    df['predict_category_property'] = df[df.predict_category_property.notnull()]['predict_category_property'].map(
        lambda x: {kv.split(':')[0]:((kv.split(':')[1].split(',') if kv.split(':')[1]!='-1' else []) if len(kv.split(':')) >= 2 else []) for kv in x.split(';')})
    return df

# 添加广告商品与查询词的相关性特征
def addContextFea(df):
    df['predict_category'] = df['predict_category_property'].dropna().map(lambda x: list(x.keys()))
    df['predict_cate_num'] = df['predict_category'].dropna().map(lambda x: len(x))
    idx = df[df.predict_category_property.notnull()].index
    df.loc[idx,'cate_intersect_num'] = list(map(lambda x: len(np.intersect1d(x[0],x[1])), df.loc[idx, ['item_category_list','predict_category']].values))
    df['predict_property'] = [set() for i in range(len(df))]
    idx = df[(df.item_category2.notnull())&(df.predict_category_property.notnull())].index
    df.loc[idx,'predict_property'] = list(map(lambda x: x[2]|set(x[1][x[0]]) if (x[0] in x[1].keys()) else x[2], df.loc[idx,['item_category2','predict_category_property','predict_property']].values))
    idx = df[(df.item_category1.notnull())&(df.predict_category_property.notnull())].index
    df.loc[idx,'predict_property'] = list(map(lambda x: x[2]|set(x[1][x[0]]) if (x[0] in x[1].keys()) else x[2], df.loc[idx,['item_category1','predict_category_property','predict_property']].values))
    df['predict_property'] = df['predict_property'].map(lambda x: np.nan if len(x)==0 else list(x))
    df['predict_prop_num'] = df[df.predict_property.notnull()]['predict_property'].map(lambda x: len(x))
    idx = df[(df.predict_property.notnull())&(df.item_property_list.notnull())].index
    df.loc[idx, 'prop_intersect_num'] = list(map(lambda x: len(np.intersect1d(x[0],x[1])), df.loc[idx, ['item_property_list','predict_property']].values))
    df.loc[idx,'prop_union_num'] = list(map(lambda x: len(np.union1d(x[0],x[1])), df.loc[idx, ['item_property_list','predict_property']].values))
    df['prop_jaccard'] = df['prop_intersect_num'] / df['prop_union_num']
    df['prop_predict_ratio'] = df['prop_intersect_num'] / df['predict_prop_num']
    df['prop_item_ratio'] = df['prop_intersect_num'] / df['item_prop_num']
    df.fillna({k:-1 for k in ['predict_prop_num','prop_intersect_num','prop_union_num','prop_jaccard','prop_predict_ratio','prop_item_ratio']}, inplace=True)
    return df



In [56]:
print(train_df_1.columns.values)


['instance_id' 'item_id' 'item_brand_id' 'item_city_id' 'item_price_level'
 'item_sales_level' 'item_collected_level' 'item_pv_level' 'user_id'
 'user_gender_id' 'user_age_level' 'user_occupation_id' 'user_star_level'
 'context_id' 'context_timestamp' 'context_page_id' 'shop_id'
 'shop_review_num_level' 'shop_review_positive_rate' 'shop_star_level'
 'shop_score_service' 'shop_score_delivery' 'shop_score_description'
 'is_trade' 'date' 'weekday' 'day' 'hour' 'match_category_proportion'
 'match_property_proportion' 'predict_category_number'
 'predict_property_number' 'isFirstCategoryIn' 'isLastCategoryIn'
 'category_number' 'property_number' 'all_item_brand_id_click_number'
 'all_item_brand_id_buy_number' 'history_item_brand_id_rate'
 'history_item_brand_id_smooth_rate' 'all_shop_id_click_number'
 'all_shop_id_buy_number' 'history_shop_id_rate'
 'history_shop_id_smooth_rate' 'all_item_id_click_number'
 'all_item_id_buy_number' 'history_item_id_rate'
 'history_item_id_smooth_rate' 'lastOn

In [57]:
train_df_3 = pd.read_csv('~/kengkeng/alimama/data/train_df_3.csv')
print(train_df_3.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1077175 entries, 0 to 1077174
Data columns (total 31 columns):
instance_id                  1077175 non-null int64
item_id                      1077175 non-null int64
item_category_list           1077175 non-null object
item_property_list           1077175 non-null object
item_brand_id                1077175 non-null int64
item_city_id                 1077175 non-null int64
item_price_level             1077175 non-null int64
item_sales_level             1077175 non-null int64
item_collected_level         1077175 non-null int64
item_pv_level                1077175 non-null int64
user_id                      1077175 non-null int64
user_gender_id               1077175 non-null int64
user_age_level               1077175 non-null int64
user_occupation_id           1077175 non-null int64
user_star_level              1077175 non-null int64
context_id                   1077175 non-null int64
context_timestamp            1077175 non-null int64
c

In [58]:
train_df_3 = splitMultiFea(train_df_3)
train_df_3 = addContextFea(train_df_3)


In [65]:
def addSet(setList):
    allSet = []
    for member in setList:
        allSet = allSet + member
    allSet = set(allSet)
    return allSet

#处理跟商品类目和属性相关的特征
def getCategoryFuture(df):
    df['predict_category_list'] = df['predict_category_property'].map(lambda x: [] if x == np.nan else list(kv.split(':')[0] for kv in str(x).split(';')))
    df['predict_category_set'] = df['predict_category_list'].map(lambda x: set(x))
    df['real_item_category_list'] = df['item_category_list'].map(lambda x: set(x))

    df['predict_property_list'] = df['predict_category_property'].map(lambda x: [] if x == np.nan else ((kv.split(':')[1].split(',') if kv.split(':')[1]!='-1' else []) if len(kv.split(':')) >= 2 else [] for kv in str(x).split(';')))
    df['predict_property_list'] = df['predict_property_list'].map(lambda x: addSet(x))
    df['item_property_list'] = df['item_property_list'].map(lambda x: set(x))
    return df

def getMatchProportion(df):
    match_category_proportion = []
    match_property_proportion = []
    for x,y,m,n in df[['real_item_category_list', 'predict_category_set', 'item_property_list', 'predict_property_list']].values:
        match_category = x & y
        match_property = m & n
        if len(y) > 0:
            category_proportion = len(match_category) / len(y)
            match_category_proportion.append(category_proportion)
        else:
            match_category_proportion.append(0)
        if len(n) > 0:
            property_proportion = len(match_property) / len(n)
            match_property_proportion.append(property_proportion)
        else:
            match_property_proportion.append(0)
    df['match_category_proportion'] = match_category_proportion
    df['match_property_proportion'] = match_property_proportion
    return df

#构造跟预测数目相关的特征
def getPredictNumber(df):
    df['predict_category_number'] = df['predict_category_set'].map(lambda x: len(x))
    df['predict_property_number'] = df['predict_property_list'].map(lambda x: len(x))
    return df

#构造跟类目预测精确性相关的特征
def getPredictAccuracy(df):
    isFirstCategoryIn = []
    isLastCategoryIn = []
    for x,y in df[['real_item_category_list', 'predict_category_list']].values:
        if y[0] in x:
            isFirstCategoryIn.append(1)
        else:
            isFirstCategoryIn.append(0)
        if y[len(y)-1] in x:
            isLastCategoryIn.append(1)
        else:
            isLastCategoryIn.append(0)
    df['isFirstCategoryIn'] = isFirstCategoryIn
    df['isLastCategoryIn'] = isLastCategoryIn
    return df

#添加商品属性个数以及类目个数特征
def getCPNumber(df):
    df['category_number'] = df['item_category_list'].map(lambda x: len(x))
    df['property_number'] = df['item_property_list'].map(lambda x: len(x))
    return df



In [60]:
train_df_3 = getCategoryFuture(train_df_3)

In [61]:
train_df_3 = getMatchProportion(train_df_3)

In [62]:
train_df_3 = getPredictNumber(train_df_3)

In [63]:
train_df_3 = getPredictAccuracy(train_df_3)

In [66]:
train_df_3 = getCPNumber(train_df_3)